# Data scraping & dataset creation

CS685 Spring 2022 <br />
Feb. 19, 2022<br />
Hongyu Tu <br />

In [3]:
import re
import time
import pickle
import asyncio
import datetime
import requests
import numpy as np
import pandas as pd 
import nest_asyncio
from utils import *
from tqdm import tqdm
from os import listdir
from collections import Counter
from bilibili_api import video, sync

nest_asyncio.apply()

In [4]:
cat_dic = init_category_dic()
tid_lst = np.array(list(cat_dic.keys()))
print('Category list initialized successfully\nWe have {} categories'.format(len(tid_lst)))

Category list initialized successfully
We have 126 categories


In [5]:
for curr_id in tqdm(tid_lst):
    danmu_lst, comment_lst = process_category(curr_id)
    for idx, name in enumerate(['danmu', 'comment']):
        fname = 'pickle2/{}/part{}.pkl'.format(name, curr_id)
        with open(fname, 'wb') as f:
            pickle.dump(danmu_lst if idx == 0 else comment_lst, f)
        f.close()

  0%|          | 0/126 [00:01<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = [[], []]
for idx, name in enumerate(['danmu', 'comment']):
    for i in listdir('pickle/{}'.format(name)):
        if 'pkl' in i:
            data[idx] = data[idx] + pickle.load(open('pickle/{}/{}'.format(name, i), 'rb'))

In [ ]:
df = list_to_csv(data[0], 'danmu')
df2 = list_to_csv(data[1], 'comment')

In [ ]:
df[-20:]

In [ ]:
df2[-20:]